In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag, log, col, mean, variance, randn, exp, lit
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('EndSen8').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/27 11:18:59 WARN Utils: Your hostname, Karthikeya, resolves to a loopback address: 127.0.1.1; using 172.25.191.235 instead (on interface zt4homnczt)
25/10/27 11:18:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/27 11:19:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/Week9/datasets/stockdata_5/AAME.csv', inferSchema=True, header=True)

In [4]:
df.show(5)

+---------+----+----+----+-----+------+
|     Date|Open|High| Low|Close|Volume|
+---------+----+----+----+-----+------+
|31-Dec-13|4.04|4.13|3.96| 4.09| 30735|
|30-Dec-13|4.05|4.05|3.84|  3.9| 14646|
|27-Dec-13|4.02|4.05|3.99| 4.05|  5047|
|26-Dec-13|3.99|4.04|3.70| 4.01|  6309|
|24-Dec-13|3.90|3.97|3.84| 3.95| 13592|
+---------+----+----+----+-----+------+
only showing top 5 rows


In [15]:
window = Window.orderBy('Date')

In [17]:
df = df.withColumn('prev_close', lag('Close').over(window)).withColumn('PDR',log(col('close') / col('prev_close')))

In [18]:
df.show(5)

25/10/27 11:03:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:03:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:03:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+----+----+----+-----+------+----------+--------------------+
|    Date|Open|High| Low|Close|Volume|prev_close|                 PDR|
+--------+----+----+----+-----+------+----------+--------------------+
|1-Apr-02|2.25|2.25|2.25| 2.25|   200|      NULL|                NULL|
|1-Apr-03|1.40|1.60|1.40| 1.55| 18500|      2.25| -0.3726752852851735|
|1-Apr-04|3.05|3.05|3.05| 3.05|  3500|      1.55|  0.6768866596881649|
|1-Apr-05|3.01|3.03|3.00| 3.03|  5655|      3.05|-0.00657897109804...|
|1-Apr-08|1.54|1.56|1.54| 1.56|  8964|      3.03|  -0.663876798259832|
+--------+----+----+----+-----+------+----------+--------------------+
only showing top 5 rows


In [24]:
stats = df.select(mean(col('PDR')), variance(col('PDR'))).collect()[0]

25/10/27 11:05:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:05:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:05:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:05:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [43]:
meani, variance = stats['avg(PDR)'], stats['variance(PDR)']
stdev = variance ** 0.5

In [29]:
drift = mean - (variance/2) 

In [35]:
df = df.withColumn('random_value', randn() * stdev)
df = df.withColumn('predicted_prec', col('close') + exp(col('random_value') + lit(drift)))

In [36]:
df.show()

25/10/27 11:11:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:11:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:11:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:11:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:11:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+----+----+----+-----+------+----------+--------------------+--------------------+------------------+
|    Date|Open|High| Low|Close|Volume|prev_close|                 PDR|        random_value|    predicted_prec|
+--------+----+----+----+-----+------+----------+--------------------+--------------------+------------------+
|1-Apr-02|2.25|2.25|2.25| 2.25|   200|      NULL|                NULL| 0.18409525522876438| 3.297583237343314|
|1-Apr-03|1.40|1.60|1.40| 1.55| 18500|      2.25| -0.3726752852851735| -0.5940079933011226|2.0311301727773636|
|1-Apr-04|3.05|3.05|3.05| 3.05|  3500|      1.55|  0.6768866596881649| 0.09653872770004912| 4.009761277212486|
|1-Apr-05|3.01|3.03|3.00| 3.03|  5655|      3.05|-0.00657897109804...| -0.4268670998804241|3.5986577483396336|
|1-Apr-08|1.54|1.56|1.54| 1.56|  8964|      3.03|  -0.663876798259832|  0.9613055441852384|3.8389074088631934|
|1-Apr-09|0.56|0.58|0.46| 0.46| 24383|      1.56|  -1.221214610760442|  0.3071991563510861|1.6448186603507482|
|

In [16]:
def RunMonteCarlo(df, n):
    window = Window.orderBy('Date')
    df = df.withColumn('prev_close', lag('Close').over(window)).withColumn('PDR',log(col('close') / col('prev_close')))
    stats = df.select(mean(col('PDR')), variance(col('PDR'))).collect()[0]
    meani, variancei = stats['avg(PDR)'], stats['variance(PDR)']
    stdev = variancei ** 0.5
    df = df.withColumn('predicted_prec', lit(0))

    # df = df.na.drop(how="any", subset=['prev_close'])

    drift = meani - (variancei/2) 

    i = 0
    while i < n:
        df = df.withColumn('random_value', randn() * stdev)
        df = df.withColumn('predicted_prec', col('predicted_prec') + col('close') + exp(col('random_value') + lit(drift)))
        i += 1

    df = df.withColumn('predicted_prec', col('predicted_prec') / lit(n))

    return df

In [17]:
df = RunMonteCarlo(df, 10)

25/10/27 11:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [18]:
df.show(5)

25/10/27 11:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/10/27 11:24:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+----+----+----+-----+------+----------+--------------------+------------------+--------------------+
|    Date|Open|High| Low|Close|Volume|prev_close|                 PDR|    predicted_prec|        random_value|
+--------+----+----+----+-----+------+----------+--------------------+------------------+--------------------+
|1-Apr-02|2.25|2.25|2.25| 2.25|   200|      NULL|                NULL|3.1576076867902563|-0.13184621428277143|
|1-Apr-03|1.40|1.60|1.40| 1.55| 18500|      2.25| -0.3726752852851735| 2.555102104385118|-0.47692598725002183|
|1-Apr-04|3.05|3.05|3.05| 3.05|  3500|      1.55|  0.6768866596881649| 4.034509909858855|  0.1491123920057115|
|1-Apr-05|3.01|3.03|3.00| 3.03|  5655|      3.05|-0.00657897109804...|  3.83585620256643| 0.04048358944493667|
|1-Apr-08|1.54|1.56|1.54| 1.56|  8964|      3.03|  -0.663876798259832|2.6161722333548054| 0.19763031331233227|
+--------+----+----+----+-----+------+----------+--------------------+------------------+--------------------+
o

In [19]:
spark.stop()